In [ ]:
!pip install -U sentence-transformers 
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import numpy as np
import sys
import pandas as pd
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertModel, TapasTokenizer, TapasForSequenceClassification
import jsonlines
import time
import os
from sentence_transformers import SentenceTransformer, util
from typing import List, Dict

# Set tokenizers parallelism to avoid warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# Load SBERT model for scoring columns
column_model = SentenceTransformer("all-MiniLM-L6-v2")  # ~22MB, fast and good

In [4]:
# Load pretrained BERT for scoring rows
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
row_model = BertModel.from_pretrained("bert-base-uncased")
row_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [28]:
# Load TAPAS in classification mode
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-tabfact")
model = TapasForSequenceClassification.from_pretrained("google/tapas-base-finetuned-tabfact")
model.eval()

tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

TapasForSequenceClassification(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.07, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0-11): 12 x TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768

In [5]:
def fetch_chunks(table_id):
  # Read chunks from chunks.json that match the target table
    print("Loading and filtering chunks for target table...")
    # chunks = []
    row_chunks = []
    column_chunks = []
    chunk_count = 0
    matched_chunks = 0

    with jsonlines.open('chunks.json', 'r') as reader:
        for chunk in reader:
            chunk_count += 1

            # Check if the chunk belongs to our target table or any of its alternative IDs
            if 'metadata' in chunk and 'table_name' in chunk['metadata'] and chunk['metadata']['table_name'] in table_id:
                # chunks.append(chunk)
                if chunk['metadata']['chunk_type'] == 'row':
                    row_chunks.append(chunk)
                else:
                    column_chunks.append(chunk)
                matched_chunks += 1

    print(f"Found {matched_chunks} chunks that match table ID '{table_id}' out of {chunk_count} total chunks")

    return row_chunks, column_chunks

In [6]:
# def fetch_column_headers(column_chunks):
#     column_headers = []
#     for chunk in column_chunks:
#         # if chunk.get('metadata', {}).get('chunk_type') == 'column':
#         metadata_text = chunk['metadata'].get('metadata_text', '')

#         # Extract 'col: ...' using simple string split
#         for part in metadata_text.split(','):
#             part = part.strip()
#             if part.startswith('col:'):
#                 col_name = part.replace('col:', '').strip()
#                 column_headers.append(col_name)
#                 break  # once 'col:' is found, we can move to next chunk

#     print("Extracted column names:")
#     print(column_headers)
#     return column_headers

In [35]:
# def fetch_column_headers_with_values(column_chunks):
#     """
#     Extracts column headers and a single value (first non-empty) per column chunk.

#     Args:
#         column_chunks (list): List of column chunks generated by `chunk_column`.

#     Returns:
#         dict: {column_name: one_value_or_dummy}
#     """
#     column_data = {}

#     for chunk in column_chunks:
#         metadata_text = chunk['metadata'].get('metadata_text', '')
#         chunk_text = chunk.get('text', '')

#         # Extract column name from metadata_text
#         col_name = None
#         for part in metadata_text.split(','):
#             part = part.strip()
#             if part.startswith('col:'):
#                 col_name = part.replace('col:', '').strip()
#                 break

#         if col_name:
#             # Default value
#             value = "dummy"

#             # Extract values after the colon
#             if ":" in chunk_text:
#                 try:
#                     _, value_part = chunk_text.split(":", 1)
#                     values = [v.strip() for v in value_part.split("|") if v.strip()]
#                     if values:
#                         value = values[0]  # Take the first non-empty value
#                 except Exception:
#                     pass

#             column_data[col_name] = value

#     print("Extracted column/value pairs:")
#     print(column_data)
#     return column_data

In [36]:
# def score_column_headers_sbert(query, column_headers):
#     """
#     Compute cosine similarity between the query and each column header using SBERT.

#     Args:
#         query (str): natural language query.
#         column_headers (List[str]): list of column names to score.

#     Returns:
#         List of tuples: (column_name, similarity_score), sorted descending.
#     """
#     # Encode query and headers
#     query_emb = column_model.encode(query, convert_to_tensor=True)
#     header_embs = column_model.encode(column_headers, convert_to_tensor=True)

#     # Compute cosine similarity
#     cosine_scores = util.cos_sim(query_emb, header_embs)[0]  # shape: [num_headers]

#     # Zip and sort
#     scored_headers = list(zip(column_headers, cosine_scores.tolist()))
#     scored_headers.sort(key=lambda x: x[1], reverse=True)

#     return scored_headers

In [45]:
# def score_column_headers_with_tapas(query: str, column_data: dict) -> float:
#     """
#     Scores each column individually for relevance to the query using TAPAS classification mode.

#     Args:
#         query (str): Natural language question.
#         column_data (dict): {column_name: one_value_or_dummy}

#     Returns:
#         dict: {column_name: relevance_score (float)}
#     """
#     results = []

#     for col, value in column_data.items():
#         df = pd.DataFrame([{col: value}])  # Create single-column, single-row DataFrame

#         try:
#             # Tokenize input
#             inputs = tokenizer(table=df, queries=[query], return_tensors="pt", truncation=True)

#             # Run model
#             with torch.no_grad():
#                 outputs = model(**inputs)
#                 logits = outputs.logits  # shape: [1, 2]
#                 probs = torch.sigmoid(logits)
#                 relevance_score = round(probs[0, 0].item(), 4)  # entailment class

#             results.append((col, relevance_score))

#         except Exception as e:
#             print(f"Error scoring column '{col}': {e}")
#             results.append((col, None))

#     return results

In [49]:
def score_column_chunks_with_sbert(query: str, column_chunks: List[dict]):
    """
    Scores each column chunk against a query using SBERT.

    Args:
        query (str): The natural language question.
        column_chunks (list): List of column chunks (from chunk_column()).

    Returns:
        List[Tuple[str, float]]: [(column_name, similarity_score)] sorted by score descending.
    """
    scored_columns = []

    # Precompute query embedding
    query_embedding = column_model.encode(query, convert_to_tensor=True)

    for chunk in column_chunks:
        metadata_text = chunk.get("metadata", {}).get("metadata_text", "")
        chunk_text = chunk.get("text", "")

        # Extract column name
        col_name = None
        for part in metadata_text.split(','):
            part = part.strip()
            if part.startswith('col:'):
                col_name = part.replace('col:', '').strip()
                break

        if not col_name:
            continue

        # Compute similarity between query and full column text (header + sample values)
        column_embedding = column_model.encode(chunk_text, convert_to_tensor=True)
        score = util.pytorch_cos_sim(query_embedding, column_embedding).item()
        score = round((score + 1) / 2, 4)  # normalize cosine similarity to [0, 1]

        scored_columns.append((col_name, score))

    # Sort by score descending
    scored_columns.sort(key=lambda x: x[1], reverse=True)

    return scored_columns

In [8]:
def filter_chunks_by_column_score(chunks, scored_columns, threshold):
    """
    Filters column chunks from the list of chunks based on precomputed SBERT scores.

    Args:
        chunks (list): List of all chunk dictionaries.
        scored_columns (list of tuples): List of (column_name, score) tuples from SBERT.
        threshold (float): Minimum score required to retain a chunk.

    Returns:
        filtered_chunks (list): Chunks whose column name score >= threshold.
    """
    # Build a set of column names to keep
    valid_columns = {col for col, score in scored_columns if score >= threshold}

    # Filter chunks that are 'column' type and have a matching valid column name
    filtered_chunks = []
    for chunk in chunks:
        metadata_text = chunk["metadata"].get("metadata_text", "")
        for part in metadata_text.split(','):
            part = part.strip()
            if part.startswith("col:"):
                col_name = part.replace("col:", "").strip()
                if col_name in valid_columns:
                    filtered_chunks.append(chunk)
                break

    return filtered_chunks

In [9]:
# @torch.no_grad()
# def score_row_tabert(chunks: List[Dict], query: str) -> List[Dict]:
#     """
#     Scores each row chunk based on its semantic similarity to the query using BERT (as a proxy for TaBERT).

#     Args:
#         chunks (List[Dict]): List of row chunks with 'text' and 'metadata' keys
#         query (str): Natural language question

#     Returns:
#         List[Dict]: List of dicts with chunk_id and relevance score
#     """
#     scores = []

#     # Encode the query once
#     query_input = tokenizer(query, return_tensors="pt", truncation=True, padding=True)
#     query_output = row_model(**query_input)
#     query_embedding = query_output.last_hidden_state[:, 0, :]  # CLS token

#     for chunk in chunks:
#         chunk_text = chunk["text"]
#         chunk_id = chunk["metadata"]["chunk_id"]

#         # Encode the chunk
#         chunk_input = tokenizer(chunk_text, return_tensors="pt", truncation=True, padding=True)
#         chunk_output = row_model(**chunk_input)
#         chunk_embedding = chunk_output.last_hidden_state[:, 0, :]  # CLS token

#         # Compute cosine similarity
#         sim = torch.nn.functional.cosine_similarity(query_embedding, chunk_embedding).item()
#         sim_normalized = (sim + 1) / 2  # normalize to [0,1]

#         scores.append({
#             "chunk_id": chunk_id,
#             "score": round(sim_normalized, 4),
#         })

#     return scores

In [23]:
def score_row_chunks_with_sbert(row_chunks: List[Dict], query: str) -> List[Dict]:
    """
    Scores each row chunk using SBERT based on semantic similarity to the query.

    Args:
        row_chunks (List[Dict]): List of chunks in your custom format, with 'text' and 'metadata'
        query (str): The input natural language query

    Returns:
        List[Dict]: List of dicts with chunk_id and similarity score (normalized 0–1)
    """
    # Pre-encode the query
    query_emb = column_model.encode(query, convert_to_tensor=True)

    scored_chunks = []
    for chunk in row_chunks:
        chunk_text = chunk.get("text", "")
        chunk_id = chunk.get("metadata", {}).get("chunk_id", "")

        # Encode chunk text and compute cosine similarity
        chunk_emb = column_model.encode(chunk_text, convert_to_tensor=True)
        sim_score = util.pytorch_cos_sim(query_emb, chunk_emb).item()

        # Normalize cosine similarity from [-1, 1] → [0, 1]
        normalized_score = round((sim_score + 1) / 2, 4)

        scored_chunks.append({
            "chunk_id": chunk_id,
            "score": normalized_score
        })

    return scored_chunks

In [10]:
def filter_chunks_by_row_score(chunks, scored_rows, threshold):
    """
    Filters row chunks from the list of chunks based on precomputed semantic similarity scores.

    Args:
        chunks (list): List of all chunk dictionaries (from chunk_row).
        scored_rows (list of dict): List of {'chunk_id': str, 'score': float} dicts.
        threshold (float): Minimum score required to retain a chunk.

    Returns:
        filtered_chunks (list): Chunks whose score >= threshold.
    """
    # Build a set of valid chunk_ids to retain
    valid_ids = {row['chunk_id'] for row in scored_rows if row['score'] >= threshold}

    # Filter chunks based on matching chunk_id
    filtered_chunks = [
        chunk for chunk in chunks
        if chunk['metadata']['chunk_id'] in valid_ids
    ]

    return filtered_chunks

In [57]:
def adaptive_alpha(scores, base=0.5, min_alpha=0.1, max_alpha=1.0):
    std_dev = np.std(scores)
    normalized_std = np.clip(std_dev / 0.2, 0, 1)  # 0.2 is a tunable baseline
    alpha = min_alpha + (max_alpha - min_alpha) * normalized_std
    return alpha

In [61]:
def dynamic_threshold(scores, alpha):
    """
    Calculate a dynamic threshold using mean + alpha * std deviation.

    Args:
        scores (list or np.array): List of similarity scores (floats between 0 and 1).
        alpha (float): Multiplier for standard deviation (default 0.5).

    Returns:
        float: Threshold value
    """
    # alpha = adaptive_alpha(scores)
    # scores = np.array(scores)
    
    # if len(scores) == 0:
    #     raise ValueError("Score list is empty.")
    
    # mean = np.mean(scores)
    # std_dev = np.std(scores)
    # threshold = mean + alpha * std_dev

    # return threshold
    median = np.median(scores)
    std_dev = np.std(scores)
    return median + alpha * std_dev

In [59]:
def column_chunks_to_dataframe(column_chunks):
    """
    Converts a list of column chunks into a pandas DataFrame.

    Args:
        column_chunks (list): List of chunks, each containing a column in 'text' and metadata.

    Returns:
        pd.DataFrame: Structured DataFrame with headers and rows.
    """
    data = {}

    for chunk in column_chunks:
        text = chunk.get("text", "")

        # Only process if the format is correct
        if ":" in text:
            header, values_str = text.split(":", 1)
            header = header.strip()

            if not header:
                continue

            # Split values and remove empty strings
            values = [v.strip() for v in values_str.strip().split("|") if v.strip()]
            data[header] = values

    # Normalize column lengths by padding with empty strings
    max_len = max((len(vals) for vals in data.values()), default=0)
    for header, values in data.items():
        if len(values) < max_len:
            values.extend([""] * (max_len - len(values)))

    # Convert to DataFrame
    df = pd.DataFrame.from_dict(data, orient='columns')

    return df

In [13]:
def row_chunks_to_dataframe(row_chunks):
    """
    Converts row-based chunks with inline 'Header: Value' format into a structured DataFrame.

    Args:
        row_chunks (list): List of row-type chunk dicts.

    Returns:
        pd.DataFrame: Structured table with one row per chunk and appropriate columns.
    """
    rows = []
    all_columns = set()

    for chunk in row_chunks:
        text = chunk.get("text", "")
        column_names = chunk.get("metadata", {}).get("columns", [])
        row_data = dict.fromkeys(column_names, "")  # initialize with empty strings

        # Split on pipe and extract 'key: value' pairs
        for part in text.split("|"):
            if ":" in part:
                key, value = part.split(":", 1)
                key = key.strip()
                value = value.strip()
                if key in column_names:
                    row_data[key] = value

        rows.append(row_data)
        all_columns.update(column_names)

    # Build DataFrame with all columns
    df = pd.DataFrame(rows, columns=sorted(all_columns))
    return df

In [14]:
def intersect_row_and_column_dfs(df_row, df_col):
    """
    Computes row-wise intersection between a row-based DataFrame and a column-based DataFrame.

    Args:
        df_row (pd.DataFrame): DataFrame constructed from row chunks.
        df_col (pd.DataFrame): DataFrame constructed from column chunks.

    Returns:
        pd.DataFrame: Rows common to both DataFrames (intersection).
    """
    # Align columns
    common_cols = sorted(set(df_row.columns).intersection(set(df_col.columns)))
    if not common_cols:
        print("⚠️ No overlapping columns found between the DataFrames.")
        return pd.DataFrame()

    df_row_sub = df_row[common_cols].copy()
    df_col_sub = df_col[common_cols].copy()

    # Drop NA to avoid mismatch due to missing values
    df_row_sub = df_row_sub.dropna()
    df_col_sub = df_col_sub.dropna()

    # Deduplicate if necessary
    df_row_sub = df_row_sub.drop_duplicates()
    df_col_sub = df_col_sub.drop_duplicates()

    # Perform intersection
    intersected = pd.merge(df_row_sub, df_col_sub, how='inner')

    return intersected

In [72]:
def dataframe_to_json_entry(df, table_id):
    """
    Convert a pandas DataFrame into a JSON-serializable dict matching the required format.

    Args:
        df (pd.DataFrame): The pruned table.
        table_id (str): Unique table identifier.

    Returns:
        dict: A JSON-compatible dictionary.
    """

    json_entry = {
        "id": table_id,
        "table": {
            "columns": [{"text": str(col)} for col in df.columns],
            "rows": [{"cells": [{"text": str(cell)} for cell in row]} for _, row in df.iterrows()],
            "tableId": table_id,
        }
    }

    return json_entry

In [70]:
def prune_table(table_id, question):
    ### Fetch chunks for the given table ID ###
    row_chunks, column_chunks = fetch_chunks(table_id)
    columns_df = column_chunks_to_dataframe(column_chunks)
    row_df = row_chunks_to_dataframe(row_chunks)
    # print("Column DF:", columns_df)
    # print("Row DF:", row_df)

    ### Column Level Pruning ###
    # column_headers_with_values = fetch_column_headers_with_values(column_chunks)

    # column_scores = score_column_headers_sbert(question, column_headers)
    # column_scores = score_column_headers_with_tapas(question, column_headers_with_values)
    column_scores = score_column_chunks_with_sbert(question, column_chunks)
    # for col, score in column_scores:
        # print(f"  {col:15s} → {score:.4f}")

    # print(f"Relevance score for table: {column_scores:.4f}")
    
    column_scores_only = [score for _, score in column_scores]
    alpha = -0.2 #check if this can be learned
    column_threshold = dynamic_threshold(column_scores_only, alpha)
    # print("Column Threshold:", column_threshold)

    filtered_column_chunks = filter_chunks_by_column_score(column_chunks, column_scores, column_threshold)
    # print(filtered_column_chunks)

    ### Row level Pruning ###
    # row_scores = score_row_tabert(row_chunks, question)
    row_scores = score_row_chunks_with_sbert(row_chunks, question)

    # for item in row_scores:
        # print(f"{item['chunk_id']} → Score: {item['score']}")

    row_scores_only = [chunk["score"] for chunk in row_scores]
    alpha = 0.5 #check if this can be learned
    row_threshold = dynamic_threshold(row_scores_only, alpha)
    # print("Row Threshold:", row_threshold)

    filtered_row_chunks = filter_chunks_by_row_score(row_chunks, row_scores, row_threshold)

    # print(f"Computing similarity scores... done!")

    filtered_columns_df = column_chunks_to_dataframe(filtered_column_chunks)
    filtered_row_df = row_chunks_to_dataframe(filtered_row_chunks)

    # print("column DF")
    # print(filtered_columns_df)
    # print('-' * 80)
    # print("row DF")
    # print(filtered_row_df)

    pruned_df = intersect_row_and_column_dfs(filtered_row_df, filtered_columns_df)

    print('-' * 80)
    print("final DF")
    print(pruned_df)

    return pruned_df

In [77]:
# Main execution block
if __name__ == "__main__":

    # Determine which table ID to use
    query_csv = "query6_TopTables.csv"
    query_csv_df = pd.read_csv(query_csv)
    question = query_csv_df['query'][0]
    print(question)
    table_list = query_csv_df['top tables'].tolist()
    target_table_id = query_csv_df['target table'][0]
    goal_answer = query_csv_df['target answer'][0]

    # all_entries = []

    start_time = time.time()

    with open("all_pruned_tables.json", "w", encoding="utf-8") as f:
        for table_id in table_list[:10]:
    
            pruned_df = prune_table(table_id, question)
    
            if not pruned_df.empty:
                entry = dataframe_to_json_entry(pruned_df,table_id)
                f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    end_time = time.time()

    print(f"Execution time: {end_time - start_time:.4f} seconds")
    

who is the chief of the gods according to ancient greek myth
Loading and filtering chunks for target table...
Found 13 chunks that match table ID 'List of pharaohs_A3680D6D69C5E013' out of 2881668 total chunks
⚠️ No overlapping columns found between the DataFrames.
--------------------------------------------------------------------------------
final DF
Empty DataFrame
Columns: []
Index: []
Loading and filtering chunks for target table...
Found 54 chunks that match table ID 'List of historical period drama films and series set in Near Eastern and Western civilization_AA41E291AC14FC48' out of 2881668 total chunks
--------------------------------------------------------------------------------
final DF
                                    Notes on setting  \
0  Docudrama which depicts a re-enactment of the ...   
1       Egypt, probably during the reign of Ahmose I   
2  based on the novel by Mika Waltari, who adapte...   
3  Egypt, during the reign of Ramesses II, based ...   
4  Egypt, 